# ANFIS pytorch for Viscosity Prediction

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class ANFIS(nn.Module):
    def __init__(self, num_rules, input_dim):
        super(ANFIS, self).__init__()
        self.num_rules = num_rules
        self.input_dim = input_dim

        # Gaussian membership function parameters
        self.centers = nn.Parameter(torch.randn(num_rules, input_dim) * 10 + 50)  # Initialize around dataset range
        self.sigmas = nn.Parameter(torch.ones(num_rules, input_dim) * 5)  # Reasonable initial sigma

        # Linear rule parameters
        self.p = nn.Parameter(torch.randn(num_rules))
        self.q = nn.Parameter(torch.randn(num_rules))
        self.r = nn.Parameter(torch.randn(num_rules))

    def forward(self, x):
        # Gaussian membership function
        gaussian = torch.exp(-((x.unsqueeze(1) - self.centers) ** 2) / (2 * self.sigmas ** 2))
        firing_strengths = gaussian.prod(dim=-1)  # Shape: (batch_size, num_rules)

        # Avoid division by zero
        sum_firing = firing_strengths.sum(dim=1, keepdim=True)
        sum_firing[sum_firing == 0] = 1.0  # Replace zero with 1.0 to prevent nan

        # Normalize firing strengths
        normalized_strengths = firing_strengths / sum_firing

        # Compute rule outputs
        rule_outputs = self.p * x[:, 0:1] + self.q * x[:, 1:2] + self.r  # Broadcasting

        # Aggregate outputs
        output = (normalized_strengths * rule_outputs).sum(dim=1)
        return output

# Dataset
data = torch.tensor([
    [50, 1.0],
    [60, 1.2],
    [70, 1.5],
    [80, 1.7],
    [90, 2.0],
], dtype=torch.float32)
targets = torch.tensor([120, 125, 130, 135, 140], dtype=torch.float32)

# Initialize model
model = ANFIS(num_rules=4, input_dim=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Training loop
for epoch in range(10000):
    optimizer.zero_grad()
    predictions = model(data)
    loss = criterion(predictions, targets)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Example usage: Predict new data
x_new = torch.tensor([[75.0, 1.6]], dtype=torch.float32)
predicted_viscosity = model(x_new)
print(f"Predicted Viscosity for T=75, P=1.6: {predicted_viscosity.item()}")


Epoch 0, Loss: 47789.8984375
Epoch 100, Loss: 4710.3818359375
Epoch 200, Loss: 832.4027099609375
Epoch 300, Loss: 355.44500732421875
Epoch 400, Loss: 216.6623077392578
Epoch 500, Loss: 194.5498504638672
Epoch 600, Loss: 192.62339782714844
Epoch 700, Loss: 192.03756713867188
Epoch 800, Loss: 191.3979949951172
Epoch 900, Loss: 190.59402465820312
Epoch 1000, Loss: 189.5292205810547
Epoch 1100, Loss: 187.9969024658203
Epoch 1200, Loss: 185.44677734375
Epoch 1300, Loss: 179.8091583251953
Epoch 1400, Loss: 154.840576171875
Epoch 1500, Loss: 67.85276794433594
Epoch 1600, Loss: 54.515594482421875
Epoch 1700, Loss: 49.842689514160156
Epoch 1800, Loss: 41.822364807128906
Epoch 1900, Loss: 30.094959259033203
Epoch 2000, Loss: 28.763973236083984
Epoch 2100, Loss: 28.422657012939453
Epoch 2200, Loss: 28.12188720703125
Epoch 2300, Loss: 27.800683975219727
Epoch 2400, Loss: 27.451183319091797
Epoch 2500, Loss: 27.06967544555664
Epoch 2600, Loss: 26.652673721313477
Epoch 2700, Loss: 26.195825576782227